In [8]:
import json
import requests
import mysql.connector

In [9]:
coordinates = "55.7263,37.6503,55.7816,37.7864"
text_query = f""" [out: json];
    node({coordinates});
    out geom; """

In [10]:
overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"
response = requests.get(overpass_url, params={'data': text_query})
json_data = response.json()
del json_data["version"]
del json_data["generator"]
del json_data["osm3s"]
with open("output_data.json", "w") as outfile:
    json.dump(json_data, outfile)

In [11]:
db = mysql.connector.connect(
    host="localhost",
    user="shadow_admin",
    password="shadow",
    database="shadow"
)

mycursor = db.cursor(dictionary=True)

In [12]:
insert_list = []
for i in range(len(json_data["elements"])):
    if(json_data["elements"][i]["type"] == "node"):
        node_id = json_data["elements"][i]["id"]
        lat     = json_data["elements"][i]["lat"]
        lon     = json_data["elements"][i]["lon"]
        insert_list.append((node_id, lat, lon))

# commit makes changes in db (through transactions)

In [13]:
sql = "INSERT INTO nodes (id, lat, lon) VALUES (%s, %s, %s)"
val = insert_list

mycursor.executemany(sql, val)

db.commit()

# must close db otherwise it will stuck

In [ ]:
db.close()